# Build Dataset B

## Description of approach B:

The idea is to find data records of single buoys that provide all measurements [“WDIR”, “WSPD”, “WVHT”, “APD”, “MWD”, “PRES”, “ATMP”, “WTMP”, “DEWP”].

### TODO: detailed description after implementation!

In [3]:
import pickle
import pandas as pd
import script_utils as mL
import datetime
import ipywidgets as widgets
from IPython.display import display
NDBC = mL.NDBC_lib

---
# Parameter:

In [4]:
def build_UI():
    #STATIONS -----------------------------------------------------------------------------------
    # create a list of checkbox widgets for each station
    stations = []
    for station in NDBC.cleaned_stations_GOM:
        checkbox = widgets.Checkbox(value=False, description=station, disabled=False, indent=False)
        stations.append(checkbox)

    # create a GridBox layout container with two columns
    global stations_grid
    stations_grid = widgets.GridBox(stations, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))

    # wrap the GridBox inside a Box layout container with a fixed height and scrollable overflow
    stations_box = widgets.Box(children=[stations_grid], layout=widgets.Layout(height="200px", overflow="scroll"))


    #Years --------------------------------------------------------------------------------------
    # create a range slider widget for selecting a time range
    global time_range_slider
    time_range_slider = widgets.SelectionRangeSlider(
        options=list(range(1970, 2023)),  # range of years to select from
        index=(51, 52),  # initial range selected (1970-2023)
        description='Time Range:',
        orientation='horizontal',
        layout={'width': '500px'}
    )


    #NaN_Theshold--------------------------------------------------------------------------------
    # create a FloatSlider widget for a value between 0 and 1
    global nan_threshold_slider
    nan_threshold_slider = widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.01,
        readout_format='.2f',
        orientation='horizontal',
        layout={'width': '500px'},
        description=''
    )

    #Features--------------------------------------------------------------------------------
    features = []
    for feature in ["WDIR", "WSPD", "WVHT", "APD", "MWD", "PRES", "ATMP", "WTMP", "DEWP"]:
        checkbox = widgets.Checkbox(value=False, description=feature, disabled=False, indent=False)
        features.append(checkbox)

    global feature_container
    feature_container = widgets.HBox(features)

    #ERA5------------------------------------------------------------------------------------
    global era5_checkbox
    era5_checkbox = widgets.Checkbox(value=True, description="Add ERA5 data", disabled=False, indent=False)

    #Supervised:
    n_in_dropdown = widgets.Dropdown(
        options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        value=3,
        description='n_in:',
        disabled=False,
    )
    n_out_dropdown = widgets.Dropdown(
        options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        value=1,
        description='n_out:',
        disabled=False,
    )

    #Test
    test_station_dropdown = widgets.Dropdown(
        options=NDBC.cleaned_stations_GOM,
        value='41117',
        description='Test Station:',
        disabled=False,
    )

    test_year_dropdown = widgets.Dropdown(
        options=range(2022, 1970, -1),
        value=2022,
        description='Test Year:',
        disabled=False,
    )

    # Test Hours-------------------------------------------------------------------------------
    global test_hours_widget
    test_hours_widget = widgets.IntSlider(
        value=24,
        min=0,
        max=1000,
        step=1,
        description='Test Hours:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )

    filename_widget = widgets.Text(
        value='',
        placeholder='Enter filename',
        description='Filename:',
        disabled=False
    )



    left_box = widgets.VBox([
        widgets.HTML(value="<h2>Train Set:</h2>"),
        stations_box,
        time_range_slider,
        widgets.HBox([widgets.HTML(value="<b>NaN-Threshold:</b>"), nan_threshold_slider]),
        nan_threshold_slider,
        feature_container,
        era5_checkbox,
        #widgets.HBox([widgets.HTML(value="<b>Supervised:</b>"), n_in_dropdown, n_out_dropdown])
        ],
        layout=widgets.Layout(width='60%')
    )

    right_box = widgets.VBox([
        widgets.HTML(value="<h2>Test Set:</h2>"),
        test_station_dropdown,
        test_year_dropdown,
        test_hours_widget,

        widgets.HTML(value="<h2>Supervised:</h2>"),
        n_in_dropdown,
        n_out_dropdown,

        widgets.HTML(value="<h2>Filename:</h2>"),
        filename_widget

    ])

    display(widgets.HBox([left_box, right_box]))



build_UI()

# UI looks quite ok but should has no functionality :D
# TODO if find time: implement functionality + improve design

In [17]:
# Optional build files list by stations and years
# stations = [ '42001',
#              '42002',
#              '42003',
#              '42007',
#              '42012',
#              '42019',
#              '42020',
#              '42035',
#              '42036',
#              '42038',
#              '42039',
#              '42040',
#              '42041',
#              '42055']
#
# years = ['2002',
#          '2003',
#          '2004',
#          '2005',
#          '2006',
#          '2007',
#          '2008',
#          '2009',
#          '2010',
#          '2011',
#          '2012',
#          '2013',
#          '2014',
#          '2015',
#          '2016',
#          '2017',
#          '2018',
#          '2019',
#          '2020',
#          '2021',
#          '2022']
#
#
# files = []
# for year in years:
#     for station in stations:
#         files.append([station, year])
#
# files

In [18]:
files = [['42001', '2002'], ['42002', '2002'], ['42003', '2002'], ['42007', '2002'], ['42019', '2002'], ['42020', '2002'], ['42035', '2002'], ['42036', '2002'], ['42039', '2002'], ['42040', '2002'], ['42041', '2002'], ['42001', '2003'], ['42002', '2003'], ['42003', '2003'], ['42007', '2003'], ['42019', '2003'], ['42020', '2003'], ['42035', '2003'], ['42036', '2003'], ['42039', '2003'], ['42040', '2003'], ['42041', '2003'], ['42001', '2004'], ['42002', '2004'], ['42003', '2004'], ['42007', '2004'], ['42019', '2004'], ['42020', '2004'], ['42035', '2004'], ['42036', '2004'], ['42038', '2004'], ['42039', '2004'], ['42040', '2004'], ['42041', '2004'], ['42001', '2005'], ['42002', '2005'], ['42003', '2005'], ['42007', '2005'], ['42019', '2005'], ['42020', '2005'], ['42035', '2005'], ['42036', '2005'], ['42038', '2005'], ['42039', '2005'], ['42040', '2005'], ['42041', '2005'], ['42055', '2005'], ['42001', '2006'], ['42002', '2006'], ['42003', '2006'], ['42007', '2006'], ['42019', '2006'], ['42020', '2006'], ['42035', '2006'], ['42036', '2006'], ['42038', '2006'], ['42039', '2006'], ['42040', '2006'], ['42055', '2006'], ['42001', '2007'], ['42002', '2007'], ['42003', '2007'], ['42007', '2007'], ['42019', '2007'], ['42020', '2007'], ['42035', '2007'], ['42036', '2007'], ['42039', '2007'], ['42040', '2007'], ['42055', '2007'], ['42001', '2008'], ['42002', '2008'], ['42003', '2008'], ['42007', '2008'], ['42019', '2008'], ['42020', '2008'], ['42035', '2008'], ['42036', '2008'], ['42039', '2008'], ['42040', '2008'], ['42055', '2008'], ['42001', '2009'], ['42002', '2009'], ['42003', '2009'], ['42007', '2009'], ['42012', '2009'], ['42019', '2009'], ['42020', '2009'], ['42035', '2009'], ['42036', '2009'], ['42039', '2009'], ['42040', '2009'], ['42055', '2009'], ['42001', '2010'], ['42002', '2010'], ['42003', '2010'], ['42012', '2010'], ['42019', '2010'], ['42020', '2010'], ['42035', '2010'], ['42036', '2010'], ['42039', '2010'], ['42040', '2010'], ['42055', '2010'], ['42001', '2011'], ['42002', '2011'], ['42003', '2011'], ['42012', '2011'], ['42019', '2011'], ['42020', '2011'], ['42035', '2011'], ['42036', '2011'], ['42039', '2011'], ['42040', '2011'], ['42055', '2011'], ['42001', '2012'], ['42002', '2012'], ['42003', '2012'], ['42012', '2012'], ['42019', '2012'], ['42020', '2012'], ['42035', '2012'], ['42036', '2012'], ['42039', '2012'], ['42040', '2012'], ['42055', '2012'], ['42001', '2013'], ['42002', '2013'], ['42003', '2013'], ['42012', '2013'], ['42019', '2013'], ['42020', '2013'], ['42035', '2013'], ['42036', '2013'], ['42039', '2013'], ['42040', '2013'], ['42055', '2013'], ['42001', '2014'], ['42002', '2014'], ['42003', '2014'], ['42012', '2014'], ['42019', '2014'], ['42020', '2014'], ['42035', '2014'], ['42036', '2014'], ['42039', '2014'], ['42040', '2014'], ['42055', '2014'], ['42001', '2015'], ['42002', '2015'], ['42003', '2015'], ['42012', '2015'], ['42019', '2015'], ['42020', '2015'], ['42035', '2015'], ['42036', '2015'], ['42039', '2015'], ['42040', '2015'], ['42055', '2015'], ['42001', '2016'], ['42002', '2016'], ['42003', '2016'], ['42012', '2016'], ['42019', '2016'], ['42020', '2016'], ['42035', '2016'], ['42036', '2016'], ['42039', '2016'], ['42040', '2016'], ['42055', '2016'], ['42001', '2017'], ['42002', '2017'], ['42003', '2017'], ['42012', '2017'], ['42019', '2017'], ['42020', '2017'], ['42035', '2017'], ['42036', '2017'], ['42039', '2017'], ['42040', '2017'], ['42055', '2017'], ['42001', '2018'], ['42002', '2018'], ['42003', '2018'], ['42012', '2018'], ['42019', '2018'], ['42020', '2018'], ['42035', '2018'], ['42039', '2018'], ['42040', '2018'], ['42055', '2018'], ['42001', '2019'], ['42002', '2019'], ['42003', '2019'], ['42012', '2019'], ['42019', '2019'], ['42020', '2019'], ['42035', '2019'], ['42036', '2019'], ['42039', '2019'], ['42040', '2019'], ['42055', '2019'], ['42001', '2020'], ['42002', '2020'], ['42003', '2020'], ['42012', '2020'], ['42019', '2020'], ['42020', '2020'], ['42035', '2020'], ['42036', '2020'], ['42039', '2020'], ['42040', '2020'], ['42055', '2020'], ['42001', '2021'], ['42002', '2021'], ['42003', '2021'], ['42012', '2021'], ['42019', '2021'], ['42020', '2021'], ['42035', '2021'], ['42036', '2021'], ['42039', '2021'], ['42040', '2021'], ['42055', '2021'], ['42001', '2022'], ['42002', '2022'], ['42003', '2022'], ['42012', '2022'], ['42019', '2022'], ['42020', '2022'], ['42035', '2022'], ['42036', '2022'], ['42039', '2022'], ['42040', '2022'], ['42055', '2022']]

#List created in development/station_selection.ipynb

In [19]:
features =  mL.measurements     # ["WDIR", "WSPD", "WVHT", "APD", "MWD", "PRES", "ATMP", "WTMP", "DEWP"]
add_ERA5 = True
n_in=3  #CHANGED!
n_out=1
nan_threshold = 0.5    # used for feature selection

# Test data:1
station_id = "42001"
year = "2022"
num_test_hours = 24

# Save as:
filename = "GOM_buoys_since2002_nan50"

---

In [20]:
#Correction of num_test_hours:
num_test_hours = num_test_hours + n_in

In [21]:
train = mL.get_data_B(files, features, n_in, n_out, nan_threshold, add_ERA5)

from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc
from disc


In [22]:
train

""
timestamp
2002-01-01 00:00:00
2002-01-01 01:00:00
2002-01-01 02:00:00
2002-01-01 03:00:00
2002-01-01 04:00:00
...
2022-12-31 19:00:00
2022-12-31 20:00:00
2022-12-31 21:00:00


# Ensure that all records in training data are from a timestamp prior the first test timestamp

In [23]:
cutoff_timestamp = pd.to_datetime(f'{int(year)+1}-01-01 00:00:00')-pd.Timedelta(hours=num_test_hours)
cutoff_timestamp

mask = train.index > cutoff_timestamp
train = train[~mask]
train

""
timestamp
2002-01-01 00:00:00
2002-01-01 01:00:00
2002-01-01 02:00:00
2002-01-01 03:00:00
2002-01-01 04:00:00
...
2022-12-30 17:00:00
2022-12-30 18:00:00
2022-12-30 19:00:00


# Depending on the provided features, a Test set is build:

In [24]:
features = train.columns
features = [f.split('(')[0].replace('_ERA5', '') for f in features]
features = list(set(features))  #remove duplicates
features

[]

In [25]:
station_id = station_id.lower()
test = mL.get_data_A([station_id], [year],1,features ,add_ERA5).tail(num_test_hours+1)
test.columns = test.columns.str.replace(f'_{station_id}', '')
test = mL.DataProcessor.data_to_supervised(test, n_in, n_out, True)
test

Started with  2022 . Previous year took:   0.0002570152282714844 seconds
from disc
Finished downloading - now merging it together!
Started with  2022 . Previous year took:   1.1920928955078125e-06 seconds
Finished downloading - now merging it together!


""
timestamp
2022-12-30 20:00:00
2022-12-30 21:00:00
2022-12-30 22:00:00
2022-12-30 23:00:00
2022-12-31 00:00:00
2022-12-31 01:00:00
2022-12-31 02:00:00
2022-12-31 03:00:00
2022-12-31 04:00:00


# Save to disc:

In [26]:
#filename ="GOM_2"

In [27]:
data_object = {
    "files": files,
    "nan_threshold": nan_threshold,
    "features": features,
    "add_era5": add_ERA5,
    "num_test_hours": num_test_hours,
    "data_train": train,
    "data_test": test
}

# open a file for writing in binary mode
filepath = f'data/datasets/type_B/{filename}_SSUM.pickle'
with open(filepath, 'wb') as f:
    # write the object to the file using pickle.dump()
    pickle.dump(data_object, f)


current_time = datetime.datetime.now().time()
print("Current time:", current_time)
print("File successfully saved:")
print(filepath)

Current time: 16:39:23.374972
File successfully saved:
data/datasets/SSUM/GOM_buoys_since2002_nan50_SSUM.pickle


In [28]:
print("DONE!")

DONE!
